<h1> Довільний текст <h1>

In [1]:
import re

# Читаємо текст з файлу
with open('./text1', 'r', encoding='utf-8') as file:
    text = file.read()

text = text.replace("\n", "").replace("ъ", "ь").lower() 
text = text.replace("\n", "").replace("ё", "е").lower() 
text = re.sub(r'[^\w\s]', '', text)  # Видаляємо всі знаки пунктуації
text = re.sub(r'\d+', '', text)  # Видаляємо всі цифри
text = text.lower()  # Перетворюємо текст на нижній регістр
# Замінюємо послідовності пробілів на один пробіл
text = re.sub(r'\s+', ' ', text).strip()

# Зберігаємо очищений текст у новий файл
with open('./text2', 'w', encoding='utf-8') as file:
    file.write(text)

print("Текст збережено в ./text2")

Текст збережено в ./text2


Підрахунок частот букв і частот біграм в тексті що перетинаються, та  що не перетинаються

In [2]:
from collections import Counter
import re
import pandas as pd

# Функція для підрахунку частоти букв
def calculate_letter_frequency(text):
    freq = Counter(text)  
    total_letters = sum(freq.values())  
    freq_percent = {letter: count / total_letters for letter, count in freq.items()}
    return dict(sorted(freq_percent.items(), key=lambda item: item[1], reverse=True))

# Функція для підрахунку частоти біграм (з перетином та без)
def calculate_bigram_frequency(text):
    # Біграми з перетином
    bigrams_with_overlap = [text[i:i+2] for i in range(len(text) - 1)]
    total_bigrams_with_overlap = len(bigrams_with_overlap) 
    freq_with_overlap = Counter(bigrams_with_overlap)

    # Біграми без перетину
    bigrams_without_overlap = [text[i:i+2] for i in range(0, len(text) - 1, 2)]
    total_bigrams_without_overlap = len(bigrams_without_overlap)  
    freq_without_overlap = Counter(bigrams_without_overlap)

    # Перетворюємо частоти у відсоткові значення
    freq_with_overlap = {bigram: count / total_bigrams_with_overlap for bigram, count in freq_with_overlap.items()}
    freq_without_overlap = {bigram: count / total_bigrams_without_overlap for bigram, count in freq_without_overlap.items()}

    return freq_with_overlap, freq_without_overlap

# Читання очищеного тексту з файлу
with open('./text2', 'r', encoding='utf-8') as file:  
    text = file.read()

# Підрахунок частоти букв та біграмів
letter_freq = calculate_letter_frequency(text) 
bigram_freq_with_overlap, bigram_freq_without_overlap = calculate_bigram_frequency(text)

# Зберігаємо в один Excel файл
with pd.ExcelWriter("frequency_data.xlsx") as writer:
    # Частота букв
    letter_df = pd.DataFrame(list(letter_freq.items()), columns=["Буква", "Частота"])
    letter_df.to_excel(writer, sheet_name="Частота Букв", index=False)

    # Частота біграм з перетином
    bigram_with_overlap_df = pd.DataFrame(list(bigram_freq_with_overlap.items()), columns=["Біграма", "Частота"])
    bigram_with_overlap_df.to_excel(writer, sheet_name="Частота Біграм З Перетином", index=False)

    # Частота біграм без перетину
    bigram_without_overlap_df = pd.DataFrame(list(bigram_freq_without_overlap.items()), columns=["Біграма", "Частота"])
    bigram_without_overlap_df.to_excel(writer, sheet_name="Частота Біграм Без Перетину", index=False)

print("\nЧастота букв і біграм збережена у файл frequency_data.xlsx")



Частота букв і біграм збережена у файл frequency_data.xlsx


Обчислення ентропії та надлишковості тексту 

In [3]:
import math
from collections import Counter

# Функція для обчислення ентропії
def calculate_entropy(ngrams, total_count, is_bigram=False):
    entropy = 0
    for count in ngrams.values():
        probability = count / total_count
        entropy -= probability * math.log2(probability)
    
    # Якщо це біграм, поділимо на 2
    if is_bigram:
        entropy /= 2
    return entropy

# Функція для підрахунку біграм
def calculate_bigram_frequency(text, overlap=True):
    if overlap:
        # Біграми з перетином
        bigrams = [text[i:i+2] for i in range(len(text) - 1)]
    else:
        # Біграми без перетину
        bigrams = [text[i:i+2] for i in range(0, len(text) - 1, 2)]
    
    total_bigrams = len(bigrams)
    bigram_counts = Counter(bigrams)
    
    return bigram_counts, total_bigrams

# Функція для підрахунку n-грам
def extract_ngrams(text, n, overlap=True):
    if overlap:
        return [text[i:i+n] for i in range(len(text) - n + 1)]
    else:
        return [text[i:i+n] for i in range(0, len(text) - n + 1, n)]

# Функція для читання файлу
def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        print("Помилка UTF-8. Використовується інше кодування...")
        with open(file_path, 'r', encoding='windows-1252', errors='replace') as file:
            text = file.read()
    return text  

# Функція для обчислення надлишковості
def calculate_redundancy(H, N):
    H_max = math.log2(N)  
    R = 1 - (H / H_max) 
    return R

# Основна функція для обчислення ентропії та надлишковості
def main_analysis(file_path):
    text = read_text_file(file_path)
          
    # Підрахунок кількості унікальних символів (розмір алфавіту)
    unique_chars = set(text)
    N = len(unique_chars)  # Розмір алфавіту
    
    # Обчислення для монограм
    monograms = extract_ngrams(text, 1)
    if len(monograms) > 0:
        monogram_counts = Counter(monograms)
        H1 = calculate_entropy(monogram_counts, len(monograms))
        R1 = calculate_redundancy(H1, N)
        print(f"Ентропія H1 (монограми): {H1:.5f}")
        print(f"Надлишковість R1 (монограми): {R1:.5f}")
    
    # Обчислення для біграм з перетином
    bigram_counts_with_overlap, total_bigrams_with_overlap = calculate_bigram_frequency(text, overlap=True)
    H2_with_overlap = calculate_entropy(bigram_counts_with_overlap, total_bigrams_with_overlap, is_bigram=True)  # Вказуємо, що це біграми
    R2_with_overlap = calculate_redundancy(H2_with_overlap, N)
    print(f"Ентропія H2 (біграми з перетином): {H2_with_overlap:.5f}")
    print(f"Надлишковість R2 (біграми з перетином): {R2_with_overlap:.5f}")
    
    # Обчислення для біграм без перетину
    bigram_counts_without_overlap, total_bigrams_without_overlap = calculate_bigram_frequency(text, overlap=False)
    H2_without_overlap = calculate_entropy(bigram_counts_without_overlap, total_bigrams_without_overlap, is_bigram=True)  # Вказуємо, що це біграми
    R2_without_overlap = calculate_redundancy(H2_without_overlap, N)
    print(f"Ентропія H2 (біграми без перетину): {H2_without_overlap:.5f}")
    print(f"Надлишковість R2 (біграми без перетину): {R2_without_overlap:.5f}")

# Виклик основної функції
file_path = "./text2"
main_analysis(file_path)


Ентропія H1 (монограми): 4.37573
Надлишковість R1 (монограми): 0.13256
Ентропія H2 (біграми з перетином): 3.98572
Надлишковість R2 (біграми з перетином): 0.20987
Ентропія H2 (біграми без перетину): 3.98463
Надлишковість R2 (біграми без перетину): 0.21009


<h1> Текст без пробілів <h1>

 Видалення пробілів із тексту

In [4]:
def remove_all_spaces(input_file, output_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            text = infile.read()

        # Видалення всіх пробілів
        text_without_spaces = text.replace(" ", "")

        # Запис тексту у новий файл
        with open(output_file, 'w', encoding='utf-8') as outfile:
            outfile.write(text_without_spaces)

        print(f"Файл '{output_file}' успішно створено.")
    except FileNotFoundError:
        print(f"Файл '{input_file}' не знайдено.")
    except Exception as e:
        print(f"Сталася помилка: {e}")

input_file = './text2'  
output_file = './text3'  
remove_all_spaces(input_file, output_file)


Файл './text3' успішно створено.


Застосування попередніх програм до text3 - текст без пробілів 

In [5]:
from collections import Counter
import pandas as pd

# Функція для підрахунку частоти букв
def calculate_letter_frequency(text):
    freq = Counter(text)  
    total_letters = sum(freq.values())  
    freq_percent = {letter: count / total_letters for letter, count in freq.items()}
    return dict(sorted(freq_percent.items(), key=lambda item: item[1], reverse=True))

# Підрахунок частоти біграм (з перетином та без)
def calculate_bigram_frequency(text):
    # Біграми з перетином
    bigrams_with_overlap = [text[i:i+2] for i in range(len(text) - 1)]
    total_bigrams_with_overlap = len(bigrams_with_overlap) 
    freq_with_overlap = Counter(bigrams_with_overlap)

    # Біграми без перетину
    bigrams_without_overlap = [text[i:i+2] for i in range(0, len(text) - 1, 2)]
    total_bigrams_without_overlap = len(bigrams_without_overlap)  
    freq_without_overlap = Counter(bigrams_without_overlap)

    # Частоти у відсоткові значення
    freq_with_overlap = {bigram: count / total_bigrams_with_overlap for bigram, count in freq_with_overlap.items()}
    freq_without_overlap = {bigram: count / total_bigrams_without_overlap for bigram, count in freq_without_overlap.items()}

    return freq_with_overlap, freq_without_overlap

# Читання очищеного тексту
with open('./text3', 'r', encoding='utf-8') as file:  
    text = file.read()

# Підрахунок частоти букв та біграмів
letter_freq = calculate_letter_frequency(text) 
bigram_freq_with_overlap, bigram_freq_without_overlap = calculate_bigram_frequency(text)

# Збереження в Excel файл
with pd.ExcelWriter("frequency_data_spaces_del.xlsx") as writer:
    # Частота букв
    letter_df = pd.DataFrame(list(letter_freq.items()), columns=["Буква", "Частота"])
    letter_df.to_excel(writer, sheet_name="Частота Букв", index=False)

    # Частота біграм з перетином
    bigram_with_overlap_df = pd.DataFrame(list(bigram_freq_with_overlap.items()), columns=["Біграма", "Частота"])
    bigram_with_overlap_df.to_excel(writer, sheet_name="Частота Біграм З Перетином", index=False)

    # Частота біграм без перетину
    bigram_without_overlap_df = pd.DataFrame(list(bigram_freq_without_overlap.items()), columns=["Біграма", "Частота"])
    bigram_without_overlap_df.to_excel(writer, sheet_name="Частота Біграм Без Перетину", index=False)

print("\nЧастота букв і біграм збережена у файл frequency_data_spaces_del.xlsx")



Частота букв і біграм збережена у файл frequency_data_spaces_del.xlsx


In [6]:
import math
from collections import Counter

# Обчислення ентропії
def calculate_entropy(ngrams, total_count, is_bigram=False):
    entropy = 0
    for count in ngrams.values():
        probability = count / total_count
        entropy -= probability * math.log2(probability)
    
    # Поділимо на 2, якщо це біграм
    if is_bigram:
        entropy /= 2
    return entropy

# Підрахунок біграм
def calculate_bigram_frequency(text, overlap=True):
    if overlap:
        # Біграми з перетином
        bigrams = [text[i:i+2] for i in range(len(text) - 1)]
    else:
        # Біграми без перетину
        bigrams = [text[i:i+2] for i in range(0, len(text) - 1, 2)]
    
    total_bigrams = len(bigrams)
    bigram_counts = Counter(bigrams)
    
    return bigram_counts, total_bigrams

# Підрахунок n-грам
def get_ngrams_from_text(text, n, overlap=True):
    if overlap:
        return [text[i:i+n] for i in range(len(text) - n + 1)]
    else:
        return [text[i:i+n] for i in range(0, len(text) - n + 1, n)]

# Читання файлу
def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        print("Помилка UTF-8. Використовується інше кодування...")
        with open(file_path, 'r', encoding='windows-1252', errors='replace') as file:
            text = file.read()
    return text  

# Обчислення надлишковості
def calculate_redundancy(H, N):
    H_max = math.log2(N)  
    R = 1 - (H / H_max) 
    return R

# Обчислення ентропії та надлишковості
def compute_entropy_and_redundancy(file_path):
    text = read_text_file(file_path)
          
    # Підрахунок кількості унікальних символів (розмір алфавіту)
    unique_chars = set(text)
    N = len(unique_chars)  # Розмір алфавіту
    
    # Обчислення для монограм
    monograms = get_ngrams_from_text(text, 1)
    if len(monograms) > 0:
        monogram_counts = Counter(monograms)
        H1 = calculate_entropy(monogram_counts, len(monograms))
        R1 = calculate_redundancy(H1, N)
        print(f"Ентропія H1 (монограми): {H1:.5f}")
        print(f"Надлишковість R1 (монограми): {R1:.5f}")
    
    # Обчислення для біграм з перетином
    bigram_counts_with_overlap, total_bigrams_with_overlap = calculate_bigram_frequency(text, overlap=True)
    H2_with_overlap = calculate_entropy(bigram_counts_with_overlap, total_bigrams_with_overlap, is_bigram=True)  # Вказуємо, що це біграми
    R2_with_overlap = calculate_redundancy(H2_with_overlap, N)
    print(f"Ентропія H2 (біграми з перетином): {H2_with_overlap:.5f}")
    print(f"Надлишковість R2 (біграми з перетином): {R2_with_overlap:.5f}")
    
    # Обчислення для біграм без перетину
    bigram_counts_without_overlap, total_bigrams_without_overlap = calculate_bigram_frequency(text, overlap=False)
    H2_without_overlap = calculate_entropy(bigram_counts_without_overlap, total_bigrams_without_overlap, is_bigram=True)  # Вказуємо, що це біграми
    R2_without_overlap = calculate_redundancy(H2_without_overlap, N)
    print(f"Ентропія H2 (біграми без перетину): {H2_without_overlap:.5f}")
    print(f"Надлишковість R2 (біграми без перетину): {R2_without_overlap:.5f}")

# Виклик основної функції
file_path = "./text3"
compute_entropy_and_redundancy(file_path)   


Ентропія H1 (монограми): 4.45070
Надлишковість R1 (монограми): 0.10986
Ентропія H2 (біграми з перетином): 4.14648
Надлишковість R2 (біграми з перетином): 0.17070
Ентропія H2 (біграми без перетину): 4.14791
Надлишковість R2 (біграми без перетину): 0.17042
